In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
#export
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import SGD

import torchvision
import torchvision.transforms as transforms


In [41]:
#export
# Data CIFAR10
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
val_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [42]:
from zero.models.resnet import *

In [43]:
model = ResNet18()

outputs = {}

def save_output(name):
    def hook(module, input, output):
        outputs[name] = output
    return hook

#
for name, module in model.named_modules():
    if list(module.children()) == []:
        module.register_forward_hook(save_output(name))

grad_outputs = {}

def save_grad_output(name):
    def hook(module, grad_input, grad_output):
        grad_outputs[name] = grad_output
    return hook

for name, module in model.named_modules():
    if list(module.children()) == []:
        module.register_backward_hook(save_grad_output(name))


In [44]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import CategoricalAccuracy, Loss

lr = 1e-3
momentum = 0.5
log_interval = 10
optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
trainer = create_supervised_trainer(model, optimizer, F.cross_entropy, device=device)
evaluator = create_supervised_evaluator(model,
                                        metrics={'nll': Loss(F.cross_entropy)},
                                        device=device)

@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(train_loader) + 1
    if iter % log_interval == 0:
        for key, value in outputs.items():
            print(key, value.data.mean().cpu().item(), value.data.std().cpu().item(),
            value.data.norm().cpu().item(), value.data.norm(p=3).cpu().item())
        for key, value in grad_outputs.items():
            print(key, value[0].data.mean().cpu().item(), value[0].data.std().cpu().item(),
            value[0].data.norm().cpu().item(), value[0].data.norm(p=3).cpu().item())
        # for name, param in model.named_parameters():
        #     print(name, param.data.mean().cpu().item(), param.data.std().cpu().item(),
        #     param.data.norm().cpu().item(), param.data.norm(p=3).cpu().item(),
        #     param.data.round().mode()[0])
        print("Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
              "".format(engine.state.epoch, iter, len(train_loader), engine.state.output))

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    avg_nll = metrics['nll']
#     for key, value in outputs.items():
#         print(key, value)
#     for key, value in grad_outputs.items():
#         print(key, value)
    print("Training Results - Epoch: {}  Avg loss: {:.2f}"
          .format(engine.state.epoch, avg_nll))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    avg_nll = metrics['nll']
    print("Validation Results - Epoch: {}  Avg loss: {:.2f}"
          .format(engine.state.epoch, avg_nll))



In [47]:
%%capture cap --no-stderr
epochs = 1
trainer.run(train_loader, max_epochs=epochs)
with open('output.txt', 'w') as f:
    f.write(cap.stdout)


qqqq
